In [1]:
from socket import gethostname
gethostname()

'hpc3-gpu-23-04'

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
from sqlalchemy import create_engine
import pandas as pd
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch
import time

In [4]:
#establish connection
engine = create_engine(os.getenv('POSTGRE_DB_URL'))

#read the dialogues table into a pandas dataframe
dialogues = pd.read_sql("SELECT * FROM dialogues WHERE dataset = 'training';", engine)
dialogues.head()

,dialogue_id,dataset,dialogue_text,actual_summary,actual_sentiment
0,3816,training,"#Person1#: Good morning, sir. Is there anythin...",#Person2# wants a suit made in 100 % black Bar...,neutral
1,3,training,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,positive
2,4,training,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,negative
3,5,training,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,positive
4,6,training,"#Person1#: Happy birthday, Aims!\n#Person2#: T...",Lisa gives Aims a birthday present and Aims lo...,positive


In [5]:
dialogues["text_length"] = dialogues["dialogue_text"].str.len()
dialogues.head()

,dialogue_id,dataset,dialogue_text,actual_summary,actual_sentiment,text_length
0,3816,training,"#Person1#: Good morning, sir. Is there anythin...",#Person2# wants a suit made in 100 % black Bar...,neutral,294
1,3,training,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,positive,500
2,4,training,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,negative,526
3,5,training,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,positive,480
4,6,training,"#Person1#: Happy birthday, Aims!\n#Person2#: T...",Lisa gives Aims a birthday present and Aims lo...,positive,302


In [6]:
index = dialogues[dialogues["dialogue_id"] == 2].index[0]
print(dialogues.iloc[index].dialogue_text)

#Person1#: Hello Mrs. Parker, how have you been?
#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.
#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.
#Person2#: What about Rubella and Mumps?
#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.
#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!
#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little.


In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
test_input = tokenizer([f"Summarize this and generate on the summary:\n{dialogues.iloc[index]["dialogue_text"]}\nSummary:"], return_tensors="pt", padding=True, truncation=True, max_length=1024).to(device)
test_output = model.generate(**test_input, max_new_tokens=200, do_sample=False, 
                                           pad_token_id=tokenizer.eos_token_id)

In [12]:
print(tokenizer.decode(test_output[0], skip_special_tokens=False))

<|begin_of_text|>Summarize this and generate on the summary:
#Person1#: Hello Mrs. Parker, how have you been?
#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.
#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.
#Person2#: What about Rubella and Mumps?
#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.
#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!
#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little.
Summary: Dr. Peters and Mrs. Parker are at the doctor's office for Ricky's vaccinations. Dr. Peters mentions that Ricky is due for Hepatitis A, Chickenpox, and Me